### --- Day 6: Chronal Coordinates ---
- Get the Points (call them destinations)
- From origin {0,0}, sort list into an ordereddict
- Define grid from the lowest to highest distinations from origin
- Create an object to represent each node that contains a destination flag and an array of nearest neighbors
- Iterate entire grid finding nearest destination to each point using Manhatten distance formula (if x=(a,b) and y=(c,d), then |a−c|+|b−d|)
- iterate entire grid, for each position add all unique nearest neighbors to a hashmap.
- display largest value

In [1]:
class Coordinate():
    def __init__(self, line):
        x = line.split(",")[0].strip()
        y = line.split(",")[1].strip()
        self.x = int(x)
        self.y = int(y)
    def __str__(self):
        return "[{0},{1}]".format(self.x,self.y)
    def __repr__(self):
        return self.__str__()

val1 = Coordinate("0,0")
val2 = Coordinate("1,1")
print(val1)
print(val2)

[0,0]
[1,1]


In [2]:
coords = open("6.txt", "r").readlines()
print(coords)

['1, 1\n', '1, 6\n', '8, 3\n', '3, 4\n', '5, 5\n', '8, 9']


In [3]:
def getManhattenDistanceBetween(val1,val2):
    if (not isinstance(val1,Coordinate) and not isinstance(val2,Coordinate)):
        print("Invalid input")
        return 
    #if debug: print(str(val1) + " & " + str(val2))
    return abs(val1.x-val2.x) + abs(val1.y-val2.y)

getManhattenDistanceBetween(Coordinate("0,0"),Coordinate("1, 1\n"))

2

In [18]:
def name_destinations(coords):
    val = 65 
    results = []
    for coord in coords:
        c = Coordinate(coord)
        result = (chr(val), c)
        results.append(result)
        val += 1
        
    return results

destinations = name_destinations(coords)
print(destinations)

[('A', [1,1]), ('B', [1,6]), ('C', [8,3]), ('D', [3,4]), ('E', [5,5]), ('F', [8,9])]


In [85]:
from pandas import * #! pip3 install pandas

dest_map = DataFrame([[0 for y in range(10)] for i in range(10)])

for name, coord in destinations:
    dest_map.loc[coord.y,coord.x] =  name     #faster, treats as one call

print(destinations)    
print(dest_map)

[('A', [1,1]), ('B', [1,6]), ('C', [8,3]), ('D', [3,4]), ('E', [5,5]), ('F', [8,9])]
   0  1  2  3  4  5  6  7  8  9
0  0  0  0  0  0  0  0  0  0  0
1  0  A  0  0  0  0  0  0  0  0
2  0  0  0  0  0  0  0  0  0  0
3  0  0  0  0  0  0  0  0  C  0
4  0  0  0  D  0  0  0  0  0  0
5  0  0  0  0  0  E  0  0  0  0
6  0  B  0  0  0  0  0  0  0  0
7  0  0  0  0  0  0  0  0  0  0
8  0  0  0  0  0  0  0  0  0  0
9  0  0  0  0  0  0  0  0  F  0


for each location on the dest_map, iterate destinations and find closest using Manhatten distance

In [84]:
def closest_match(position):
    distance = 1000
    match_destination = None
    for destination in destinations:
        tmp = getManhattenDistanceBetween(position,destination[1])
        
        if not match_destination is None:
            if tmp<distance:
                distance = tmp
                match_destination = destination
        else:
            match = destination
            match_destination = tmp
    return match_destination 
        
dest = closest_match(Coordinate("5,5"))    
print("Closest destination: " + str(dest))

Closest destination: ('E', [5,5])
